## Building LSTM

In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import joblib

# Load dataset
df = pd.read_csv("highrange_25yrs_indian_coalmine_data.csv")

# Feature & target columns
features = df[["Fuel_Used_Liters", "Coal_Mined_Tonnes"]].values
target = df["CO2_Emitted_kg"].values.reshape(-1, 1)

# Scaling
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_x.fit_transform(features)
y_scaled = scaler_y.fit_transform(target)

# Save scalers
joblib.dump(scaler_x, "scaler_x.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

# Create sequences
TIME_STEPS = 14

def create_sequences(X, y, time_steps=14):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

# Build Functional Model
input_layer = Input(shape=(TIME_STEPS, 2))
x = LSTM(128, return_sequences=True)(input_layer)
x = LSTM(64)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Save final model
model.save("lstm_co2_predictor.h5")
print("✅ Model and scalers saved successfully!")

Epoch 1/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0266 - mae: 0.1061 - val_loss: 0.0053 - val_mae: 0.0576
Epoch 2/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0050 - mae: 0.0566 - val_loss: 0.0053 - val_mae: 0.0584
Epoch 3/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0049 - mae: 0.0559 - val_loss: 0.0053 - val_mae: 0.0575
Epoch 4/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0051 - mae: 0.0575 - val_loss: 0.0047 - val_mae: 0.0549
Epoch 5/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0050 - mae: 0.0558 - val_loss: 0.0047 - val_mae: 0.0546
Epoch 6/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0047 - mae: 0.0546 - val_loss: 0.0048 - val_mae: 0.0551
Epoch 7/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0047 - mae: 0.0548 - val_loss: 0.0048 - val_mae: 0.0558
Epoch 8/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0046 - mae: 0.0543 - val_loss: 0.0048 - val_mae: 0.0554
Epoch 9/50
205/205 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/

✅ Model and scalers saved successfully!


In [2]:
model.save("lstm_co2_predictor.h5")